In [1]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform

Using TensorFlow backend.
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
def identity_block(X, f, filters, stage, block):
	conv_name_base = 'res' + str(stage) + block + '_branch'
	bn_name_base = 'bn' + str(stage) + block + '_branch'

	F1, F2, F3 = filters

	X_shortcut = X

	X = Conv2D(F1, (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
	X = Activation('relu')(X)

	X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
	X = Activation('relu')(X)

	X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


	X = layers.Add()([X, X_shortcut])
	X = Activation('relu')(X)

	return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
	conv_name_base = 'res' + str(stage) + block + '_branch'
	bn_name_base = 'bn' + str(stage) + block + '_branch'

	F1, F2, F3 = filters

	X_shortcut = X

	X = Conv2D(F1, (1,1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
	X = Activation('relu')(X)

	X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
	X = Activation('relu')(X)

	X = Conv2D(F3, (1,1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

	X_shortcut = Conv2D(F3, (1,1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
	X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

	X = layers.Add()([X, X_shortcut])
	X = Activation('relu')(X)

	return X

In [4]:
input_shape = (64, 64, 3)

In [5]:
classes = 6

In [6]:
# Define the input as a tensor with shape input_shape
X_input = Input(input_shape)

In [7]:
# Zero-Padding
X = ZeroPadding2D((3, 3))(X_input)

# Stage 1
X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1',)(X)
X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

# Stage 2
X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

# Stage 3
X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

# Stage 4
X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

# Stage 5
X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

# AVGPOOL
X = AveragePooling2D((2,2), name='avg_pool')(X)

# output layer
X = Flatten()(X)
X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

In [8]:
# Create model
resnet_model = Model(inputs = X_input, outputs = X, name='ResNet50')

In [9]:
#Compile the model
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [13]:
x_train=train_datagen.flow_from_directory(r"D:\SkinDisease\dataset\train_set",target_size=(64,64),batch_size=12,class_mode='sparse')

Found 1401 images belonging to 5 classes.


In [14]:
x_test=test_datagen.flow_from_directory(r"D:\SkinDisease\dataset\test_set",target_size=(64,64),batch_size=12,class_mode='sparse')

Found 329 images belonging to 5 classes.


In [15]:
resnet_model.fit_generator(x_train,samples_per_epoch=1400,epochs=20,validation_data=x_test,nb_val_samples=329)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, validation_data=<keras_pre..., steps_per_epoch=116, validation_steps=329)`
  """Entry point for launching an IPython kernel.


Epoch 1/20
116/116 [==============================] - 479s 4s/step - loss: 2.4114 - acc: 0.4095 - val_loss: 3.2868 - val_acc: 0.2415
Epoch 2/20
116/116 [==============================] - 453s 4s/step - loss: 2.5633 - acc: 0.4303 - val_loss: 14.4184 - val_acc: 0.0867
Epoch 3/20
116/116 [==============================] - 454s 4s/step - loss: 2.5545 - acc: 0.4377 - val_loss: 7.1877 - val_acc: 0.2089
Epoch 4/20
116/116 [==============================] - 461s 4s/step - loss: 2.4489 - acc: 0.4847 - val_loss: 2.9265 - val_acc: 0.3455
Epoch 5/20
116/116 [==============================] - 395s 3s/step - loss: 2.4687 - acc: 0.4681 - val_loss: 13.2771 - val_acc: 0.1064
Epoch 6/20
116/116 [==============================] - 399s 3s/step - loss: 2.4452 - acc: 0.3951 - val_loss: 7.3765 - val_acc: 0.3297
Epoch 7/20
116/116 [==============================] - 430s 4s/step - loss: 2.5258 - acc: 0.3290 - val_loss: 6.5188 - val_acc: 0.2917
Epoch 8/20
116/116 [==============================] - 419s 4s/step 

In [17]:
resnet_model.save('project-testRESNET.h5')

In [18]:
from keras.models import load_model
import numpy as np

In [19]:
import cv2

In [20]:
pred_model=load_model('project-testRESNET.h5')

In [21]:
from skimage.transform import resize

In [28]:
def detect(frame):
    try:
        img=resize(frame,(64,64))
        img=np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img=img/255.0
        prediction=resnet_model.predict(img)
        print(prediction)
        print(resnet_model.predict_classes(img))
    except AttributeError:
        print("Shape not found")

In [29]:
frame=cv2.imread(r'D:\SkinDisease\dataset\test_set\actinic keratosis\ISIC_0030825.jpg')
data=detect(frame)

[[7.9534896e-02 2.0433201e-03 1.1521480e-02 3.9227808e-01 5.1460558e-01
  1.6691061e-05]]
Shape not found


In [30]:
print(x_train.class_indices)

{'actinic keratosis': 0, 'melanoma': 1, 'nevus': 2, 'pigmented benign keratosis': 3, 'vascular lesion': 4}


In [31]:
frame2=cv2.imread(r'D:\SkinDisease\dataset\test_set\nevus\ISIC_0000563.jpg')
data=detect(frame2)

[[5.26576526e-02 1.44657232e-02 3.81377526e-02 7.72916257e-01
  1.21797405e-01 2.52051359e-05]]
Shape not found


In [33]:
frame3=cv2.imread(r'D:\SkinDisease\dataset\test_set\melanoma\ISIC_0000016.jpg')
data=detect(frame3)

[[1.8995322e-02 2.9697627e-01 3.6165050e-01 2.8129494e-01 4.1063365e-02
  1.9494142e-05]]
Shape not found


In [34]:
frame3=cv2.imread(r'D:\SkinDisease\dataset\test_set\pigmented benign keratosis\ISIC_0024324.jpg')
data=detect(frame3)

[[5.7294942e-02 9.6596952e-04 8.1747370e-03 4.3720534e-01 4.9634901e-01
  1.0056780e-05]]
Shape not found


In [35]:
frame3=cv2.imread(r'D:\SkinDisease\dataset\test_set\vascular lesion\ISIC_0032890.jpg')
data=detect(frame3)

[[7.3038898e-02 4.5880042e-03 2.4603678e-02 5.3525102e-01 3.6248714e-01
  3.1222371e-05]]
Shape not found
